<a href="https://colab.research.google.com/github/sydrafaey/restaurant-management-app/blob/main/Copy_of_Lab5_Transfer_Learning_using_Pretrained_Layers_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reusing Pretrained Layers

Reusing a Keras model
Let's split the fashion MNIST training set in two:

X_train_A: all images of all items except for sandals and shirts (classes 5 and 6).

X_train_B: a much smaller training set of just the first 200 images of sandals or shirts.

The validation set and the test set are also split, but without restricting the number of images.

We will train a model on set A (classification task with 8 classes), and try to reuse it to tackle set B (binary classification). We hope to transfer a little bit of knowledge from task A to task B, since classes in set A (sneakers, ankle boots, coats, t-shirts, etc.) are somewhat similar to classes in set B (sandals and shirts). However, since we are using Dense layers, only patterns that occur at the same location can be reused.

Load all necessary libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
# loading the Fashion-MNIST dataset
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
len(X_train_full)

60000

In [ ]:
len(X_test)

10000

In [ ]:
# trim the data 
X_train_full = X_train_full[:30000]
y_train_full = y_train_full[:30000]

X_test = X_test[:5000]
y_test = y_test[:5000]

# scaling the dataset
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0

# dividing the dataset into training and validation set
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [ ]:
len(X_train_full)

30000

In [ ]:
len(X_valid)

5000

In [ ]:
len(X_train)

25000

In [ ]:
X_train_full

array([[[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0. 

In [ ]:
# defining the dataset
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))



(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [ ]:
X_train_A.shape # checking shape of training set A

(19875, 28, 28)

In [ ]:
X_train_B.shape # checking shape of training set B

(200, 28, 28)

In [ ]:
y_train_A[:30] # checking first 30 y-labels for training set A

array([4, 0, 5, 7, 7, 7, 4, 4, 3, 4, 0, 1, 6, 3, 4, 3, 2, 6, 5, 3, 4, 5,
       1, 3, 4, 2, 0, 6, 7, 1], dtype=uint8)

In [ ]:
y_train_B[:30] # checking first 30 y-labels for training set B

array([1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.], dtype=float32)

In [ ]:
# defining the model
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [ ]:
# compiling the model
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

In [ ]:
# training the model
history = model_A.fit(X_train_A, y_train_A, epochs=5,
                    validation_data=(X_valid_A, y_valid_A))

Epoch 1/5
622/622 [==============================] - 3s 3ms/step - loss: 1.1005 - accuracy: 0.6405 - val_loss: 0.4654 - val_accuracy: 0.8411
Epoch 2/5
622/622 [==============================] - 2s 3ms/step - loss: 0.4377 - accuracy: 0.8556 - val_loss: 0.3871 - val_accuracy: 0.8625
Epoch 3/5
622/622 [==============================] - 2s 3ms/step - loss: 0.3765 - accuracy: 0.8719 - val_loss: 0.3755 - val_accuracy: 0.8652
Epoch 4/5
622/622 [==============================] - 2s 3ms/step - loss: 0.3376 - accuracy: 0.8828 - val_loss: 0.3303 - val_accuracy: 0.8857
Epoch 5/5
622/622 [==============================] - 2s 3ms/step - loss: 0.3210 - accuracy: 0.8905 - val_loss: 0.3229 - val_accuracy: 0.8859


In [ ]:
model_A.save("my_model_A.h5") # saving the model we created

In [ ]:
# defining the model
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [ ]:
# compiling the model with binary crossentropy
# that can accept either logits (i.e values from last linear node, z)
# or probabilities from the last Sigmoid node
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

In [ ]:
# training the model
history = model_B.fit(X_train_B, y_train_B, epochs=5,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/5
7/7 [==============================] - 1s 43ms/step - loss: 0.5490 - accuracy: 0.7051 - val_loss: 0.3951 - val_accuracy: 0.8854
Epoch 2/5
7/7 [==============================] - 0s 16ms/step - loss: 0.3572 - accuracy: 0.8809 - val_loss: 0.3238 - val_accuracy: 0.9229
Epoch 3/5
7/7 [==============================] - 0s 15ms/step - loss: 0.2792 - accuracy: 0.9437 - val_loss: 0.2761 - val_accuracy: 0.9412
Epoch 4/5
7/7 [==============================] - 0s 17ms/step - loss: 0.2408 - accuracy: 0.9579 - val_loss: 0.2424 - val_accuracy: 0.9523
Epoch 5/5
7/7 [==============================] - 0s 16ms/step - loss: 0.2051 - accuracy: 0.9881 - val_loss: 0.2165 - val_accuracy: 0.9604


In [ ]:
model_A.summary() # generating model summary

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 300)               235500    
_________________________________________________________________
dense_25 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_26 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_27 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_28 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_29 (Dense)             (None, 8)                

In [ ]:
model_B.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_30 (Dense)             (None, 300)               235500    
_________________________________________________________________
dense_31 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_32 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_33 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_34 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_35 (Dense)             (None, 1)                

In [ ]:
model_A = keras.models.load_model("my_model_A.h5") # loading our saved model
model_B_on_A = keras.models.Sequential(model_A.layers[:-1]) # creating new model based on existing layer
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid")) # adding new layer to new model



In [ ]:
# model_A and model_B_on_A now share some layers. When you train
# model_B_on_A, it will also affect model_A. To avoid that, you need to clone
# model_A before you reuse its layers. To do this, you clone model A’s
# architecture with clone_model(), then copy its weights
# (since clone_model() does not clone the weights)
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [ ]:
# freezing reused layers
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

# compiling the model
model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-3),
                     metrics=["accuracy"])

In [ ]:
# training the model
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

# unfreezing reused layers
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

# compiling after reducing learning rate to avoid damaging the reused weights
model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-3),
                     metrics=["accuracy"])

# training the model
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=5,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 0s 21ms/step - loss: 0.1387 - accuracy: 0.9850 - val_loss: 0.1463 - val_accuracy: 0.9787
Epoch 2/4
7/7 [==============================] - 0s 15ms/step - loss: 0.1250 - accuracy: 0.9850 - val_loss: 0.1354 - val_accuracy: 0.9787
Epoch 3/4
7/7 [==============================] - 0s 15ms/step - loss: 0.1141 - accuracy: 0.9950 - val_loss: 0.1265 - val_accuracy: 0.9797
Epoch 4/4
7/7 [==============================] - 0s 17ms/step - loss: 0.1051 - accuracy: 0.9950 - val_loss: 0.1188 - val_accuracy: 0.9817
Epoch 1/5
7/7 [==============================] - 1s 45ms/step - loss: 0.0992 - accuracy: 0.9931 - val_loss: 0.1122 - val_accuracy: 0.9838
Epoch 2/5
7/7 [==============================] - 0s 16ms/step - loss: 0.0859 - accuracy: 0.9981 - val_loss: 0.1066 - val_accuracy: 0.9838
Epoch 3/5
7/7 [==============================] - 0s 17ms/step - loss: 0.0813 - accuracy: 0.9912 - val_loss: 0.1018 - val_accuracy: 0.9838
Epoch 4/5
7/7 [===================

In [ ]:
model_B.evaluate(X_test_B, y_test_B) # evaluating the model A

31/31 [==============================] - 0s 2ms/step - loss: 0.2211 - accuracy: 0.9576


[0.22109617292881012, 0.9576008319854736]

In [ ]:
model_B_on_A.evaluate(X_test_B, y_test_B) # evaluating the model B

31/31 [==============================] - 0s 2ms/step - loss: 0.0875 - accuracy: 0.9938


[0.08754471689462662, 0.9937952160835266]